# Imports

In [1]:
from datasets import load_dataset
from transformers import (AutoModelForSeq2SeqLM, AutoTokenizer, 
                          Seq2SeqTrainingArguments, Seq2SeqTrainer)
import random
import torch

C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data

In [2]:
train_dataset = load_dataset("scientific_papers", "pubmed", split="train")

Found cached dataset scientific_papers (C:/Users/ronna/.cache/huggingface/datasets/scientific_papers/pubmed/1.1.1/306757013fb6f37089b6a75469e6638a553bd9f009484938d8f75a4c5e84206f)


In [3]:
val_dataset = load_dataset("scientific_papers", "pubmed", split="validation")

Found cached dataset scientific_papers (C:/Users/ronna/.cache/huggingface/datasets/scientific_papers/pubmed/1.1.1/306757013fb6f37089b6a75469e6638a553bd9f009484938d8f75a4c5e84206f)


In [4]:
test_dataset = load_dataset("scientific_papers", "pubmed", split="test")

Found cached dataset scientific_papers (C:/Users/ronna/.cache/huggingface/datasets/scientific_papers/pubmed/1.1.1/306757013fb6f37089b6a75469e6638a553bd9f009484938d8f75a4c5e84206f)


# Preprocess Data

## Load Tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

## Set Params

In [6]:
max_input_length = 8192
max_output_length = 512
batch_size = 2

In [7]:
def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=max_input_length)
    outputs = tokenizer(batch["abstract"], padding="max_length", truncation=True, max_length=max_output_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["global_attention_mask"] = len(batch["input_ids"]) * [[0 for _ in range(len(batch["input_ids"][0]))]]
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

## Downsample

In [8]:
num_shards = 1000
raw_sub_train_dataset = train_dataset.shard(num_shards=num_shards, index=random.randint(0, num_shards - 1))
raw_sub_val_dataset = val_dataset.shard(num_shards=num_shards, index=random.randint(0, num_shards - 1))

## Tokenize and Convert to Torch

In [9]:
sub_train_dataset = raw_sub_train_dataset.map(process_data_to_model_inputs, batched=True, batch_size=batch_size, remove_columns=["article", "abstract", "section_names"])
sub_val_dataset = raw_sub_val_dataset.map(process_data_to_model_inputs, batched=True, batch_size=batch_size, remove_columns=["article", "abstract", "section_names"])

In [10]:
sub_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "global_attention_mask", "labels"])
sub_val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "global_attention_mask", "labels"])

# Model 1

In [324]:
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)

## Train Model 1

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="allenai/led-base-16384_finetuned",
    overwrite_output_dir=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    logging_dir="allenai/led-base-16384_logs",
    num_train_epochs=3,
    save_total_limit=1,
    load_best_model_at_end=True,
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=led,
    args=training_args,
    train_dataset=sub_train_dataset,
    eval_dataset=sub_val_dataset,
    tokenizer=tokenizer,
)

trainer.train()

C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


## Test Model 1

In [ ]:
led.config.num_beams = 1
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

In [ ]:
random_index = random.randint(0, len(sub_val_dataset) - 1)
sample = sub_val_dataset[random_index]

device = torch.device("cpu")
led.to(device)

input_ids = sample["input_ids"].unsqueeze(0).to(device)
attention_mask = sample["attention_mask"].unsqueeze(0).to(device)
global_attention_mask = sample["global_attention_mask"].unsqueeze(0).to(device)

with torch.no_grad():
    summary_ids = led.generate(input_ids=input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)

generated_summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
actual_summary = tokenizer.decode(sample["labels"], skip_special_tokens=True)

# Print and compare both summaries
print("Generated Summary:")
print(generated_summary)
print("\nActual Summary:")
print(actual_summary)

# Train Variety of Models

In [ ]:
model_names = ["allenai/led-base-16384"] # "t5-small", "facebook/bart-small", "google/pegasus-small"]

for model_name in model_names:
    print(f"\nTraining {model_name}\n")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, gradient_checkpointing=True, use_cache=False)

    # Configure model settings
    model.config.num_beams = 1
    model.config.max_length = 512
    model.config.min_length = 100
    model.config.length_penalty = 2.0
    model.config.early_stopping = True
    model.config.no_repeat_ngram_size = 3

    training_args = Seq2SeqTrainingArguments(
        output_dir=f"{model_name}_finetuned",
        overwrite_output_dir=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        logging_dir=f"{model_name}_logs",
        num_train_epochs=3,
        save_total_limit=1,
        load_best_model_at_end=True,
        report_to="none",
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=sub_train_dataset,
        eval_dataset=sub_val_dataset,
        tokenizer=tokenizer,
    )

    trainer.train()

# LDA Exploration - Potential Baseline?

In [12]:
import gensim
import nltk
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ronna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Individual Article

In [13]:
article = train_dataset[0]["article"]

In [14]:
single_sentences = sent_tokenize(article)
single_word_tokenized_sentences = [word_tokenize(sent.lower()) for sent in single_sentences]

In [15]:
single_dictionary = Dictionary(single_word_tokenized_sentences)
single_corpus = [single_dictionary.doc2bow(text) for text in single_word_tokenized_sentences]

In [16]:
single_lda_model = LdaModel(single_corpus, num_topics=1, id2word=single_dictionary, passes=10)

In [18]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(single_sentences)
single_lda_importances = single_lda_model.get_document_topics(single_corpus, minimum_probability=0)
important_sentences = sorted(zip(range(len(single_sentences)), single_lda_importances), key=lambda x: -x[1][0][1])
summary_length = 7
summary_sentences = [single_sentences[i[0]].replace("\n", " ") for i in important_sentences[:summary_length]]
summary = ' '.join(summary_sentences)

In [19]:
print("Abstract:")
print(train_dataset[0]['abstract'].replace("\n", " "))

print("\nGenerated LDA Abstract:")
print(summary)

Abstract:
 background : the present study was carried out to assess the effects of community nutrition intervention based on advocacy approach on malnutrition status among school - aged children in shiraz , iran.materials and methods : this case - control nutritional intervention has been done between 2008 and 2009 on 2897 primary and secondary school boys and girls ( 7 - 13 years old ) based on advocacy approach in shiraz , iran .   the project provided nutritious snacks in public schools over a 2-year period along with advocacy oriented actions in order to implement and promote nutritional intervention . for evaluation of effectiveness of the intervention growth monitoring indices of pre- and post - intervention were statistically compared.results:the frequency of subjects with body mass index lower than 5% decreased significantly after intervention among girls ( p = 0.02 ) .   however , there were no significant changes among boys or total population .   the mean of all anthropometr

## Trained on Train

In [21]:
test_article = train_dataset[0]["article"]

In [22]:
def preprocess_data(article):
    sentences = sent_tokenize(article)
    word_tokenized_sentences = [word_tokenize(sent.lower()) for sent in sentences]
    return word_tokenized_sentences

In [23]:
lda_train_dataset = train_dataset # train_dataset.shard(num_shards=10, index=random.randint(0, 10 - 1))

In [24]:
train_corpus = []
for data in lda_train_dataset:
    article_sentences = preprocess_data(data["article"])
    train_corpus.extend(article_sentences)

dictionary = Dictionary(train_corpus)
corpus = [dictionary.doc2bow(text) for text in train_corpus]

In [25]:
num_topics = 1
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

In [101]:
def lda_get_top_n_sentences(article, lda_model, dictionary, n=10):
    sentences = sent_tokenize(article)
    word_tokenized_sentences = [word_tokenize(sent.lower()) for sent in sentences]

    sentence_topic_distributions = [lda_model.get_document_topics(dictionary.doc2bow(sentence)) for sentence in word_tokenized_sentences]
    sorted_sentences = sorted(enumerate(sentence_topic_distributions), key=lambda x: x[1][0][1], reverse=True)
    top_n_sentences_indices = [sentence_info[0] for sentence_info in sorted_sentences[:n]]
    top_n_sentences = [sentences[idx] for idx in top_n_sentences_indices]

    return top_n_sentences

In [55]:
important_sentences = lda_get_top_n_sentences(test_article, lda_model, dictionary)
summary = " ".join(important_sentences)

In [56]:
print("Abstract:")
print(train_dataset[0]['abstract'].replace("\n", " "))

print("\nGenerated LDA Abstract:")
print(summary)

Abstract:
 background : the present study was carried out to assess the effects of community nutrition intervention based on advocacy approach on malnutrition status among school - aged children in shiraz , iran.materials and methods : this case - control nutritional intervention has been done between 2008 and 2009 on 2897 primary and secondary school boys and girls ( 7 - 13 years old ) based on advocacy approach in shiraz , iran .   the project provided nutritious snacks in public schools over a 2-year period along with advocacy oriented actions in order to implement and promote nutritional intervention . for evaluation of effectiveness of the intervention growth monitoring indices of pre- and post - intervention were statistically compared.results:the frequency of subjects with body mass index lower than 5% decreased significantly after intervention among girls ( p = 0.02 ) .   however , there were no significant changes among boys or total population .   the mean of all anthropometr

In [109]:
def get_top_n_sentences(article, topic_distributions, n=10):
    sentences = sent_tokenize(article)
    sorted_sentences = sorted(enumerate(topic_distributions), key=lambda x: x[1][0], reverse=True)
    top_n_sentences_indices = [sentence_info[0] for sentence_info in sorted_sentences[:n]]
    top_n_sentences = [sentences[idx] for idx in top_n_sentences_indices]
    return top_n_sentences

# Non-negative Matrix Factorization (NMF):

In [30]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
def create_nmf_model(train_corpus, num_topics=1):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform([' '.join(text) for text in train_corpus])
    
    nmf_model = NMF(n_components=num_topics, random_state=42)
    nmf_model.fit(X)
    
    return nmf_model, vectorizer

In [32]:
nmf_model, vectorizer = create_nmf_model(train_corpus, num_topics=num_topics)

In [33]:
def get_nmf_topic_distribution(article_sentences, nmf_model, vectorizer):
    X = vectorizer.transform([' '.join(text) for text in article_sentences])
    topic_distribution = nmf_model.transform(X)
    return topic_distribution

In [34]:
nmf_topic_distribution = get_nmf_topic_distribution(preprocess_data(test_article), nmf_model, vectorizer)
important_sentences = get_top_n_sentences(test_article, nmf_topic_distribution, n=10)
nmf_summary = " ".join(important_sentences)
print("\nGenerated NMF Abstract:")
print(nmf_summary.replace('\n', ' '))


Generated NMF Abstract:
there was also a significant increase in the proportion of children with bmi that was normal for age ( 2 to + 1 sd ) most of the published community interventions showed better results among females compared with males . the mean of age in welfare group was 10.0  2.3 and 10.5  2.5 in non - welfare group . the results of the mentioned study showed an improvement in the weight of children , psychological test 's scores and the grade - point average following this school feeding program . the pre- and post - test education assessment in both groups showed that the student 's average knowledge score has been significantly increased from 12.5  3.2 to 16.8  4.3 ( p < 0.0001 ) . bmi for age for iranian students aged 7 - 14 years based on gender according to who growth standards 2007 bmi for age for iranian students aged 7 - 14 years according to who growth standards 2007 in non - welfare and welfare groups of total population table 4 has shown the prevalence of normal

# Latent Semantic Analysis (LSA) or Latent Semantic Indexing (LSI):

In [35]:
from sklearn.decomposition import TruncatedSVD

In [36]:
def create_lsa_model(train_corpus, num_topics=1):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform([' '.join(text) for text in train_corpus])

    lsa_model = TruncatedSVD(n_components=num_topics, random_state=42)
    lsa_model.fit(X)

    return lsa_model, vectorizer

In [37]:
lsa_model, vectorizer = create_lsa_model(train_corpus, num_topics=num_topics)

In [38]:
def get_lsa_topic_distribution(article_sentences, lsa_model, vectorizer):
    X = vectorizer.transform([' '.join(text) for text in article_sentences])
    topic_distribution = lsa_model.transform(X)
    return topic_distribution

In [39]:
lsa_topic_distribution = get_lsa_topic_distribution(preprocess_data(test_article), lsa_model, vectorizer)
important_sentences = get_top_n_sentences(test_article, lsa_topic_distribution, n=10)
lsa_summary = " ".join(important_sentences)
print("\nGenerated LSA Abstract:")
print(lsa_summary.replace('\n', ' '))


Generated LSA Abstract:
there was also a significant increase in the proportion of children with bmi that was normal for age ( 2 to + 1 sd ) most of the published community interventions showed better results among females compared with males . the mean of age in welfare group was 10.0  2.3 and 10.5  2.5 in non - welfare group . the results of the mentioned study showed an improvement in the weight of children , psychological test 's scores and the grade - point average following this school feeding program . the pre- and post - test education assessment in both groups showed that the student 's average knowledge score has been significantly increased from 12.5  3.2 to 16.8  4.3 ( p < 0.0001 ) . bmi for age for iranian students aged 7 - 14 years based on gender according to who growth standards 2007 bmi for age for iranian students aged 7 - 14 years according to who growth standards 2007 in non - welfare and welfare groups of total population table 4 has shown the prevalence of normal

# Hierarchical Dirichlet Process (HDP):

In [40]:
from gensim.models import HdpModel

In [41]:
hdp_model = hdp_model = HdpModel(corpus, id2word=dictionary)

In [42]:
def get_hdp_topic_distribution(article_sentences, hdp_model, dictionary):
    bow = [dictionary.doc2bow(sent) for sent in article_sentences]
    topic_distribution = [hdp_model[c] for c in bow]
    return topic_distribution

In [43]:
hdp_topic_distribution = get_hdp_topic_distribution(preprocess_data(test_article), hdp_model, dictionary)
important_sentences = get_top_n_sentences(test_article, hdp_topic_distribution, n=10)
hdp_summary = " ".join(important_sentences)
print("\nGenerated HDP Abstract:")
print(hdp_summary.replace('\n', ' '))


Generated HDP Abstract:
in general , the new snack package in average has provided 380 kcal energy , 15 g protein along with sufficient calcium and iron . in general , the new snack package in average has provided 380 kcal energy , 15 g protein along with sufficient calcium and iron . in general , the new snack package in average has provided 380 kcal energy , 15 g protein along with sufficient calcium and iron . statistical analyses were performed using the statistical package for the social sciences ( spss ) software , version 17.0 ( spss inc . for implementing the interventions , weight was to the nearest 0.1 kg on a balance scale ( model # seca scale ) . statistical analyses were performed using the statistical package for the social sciences ( spss ) software , version 17.0 ( spss inc . snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . standing height was measured to the nearest 0.1 cm with a wall - mounted stadiometer . standing height was mea

# Correlation Explanation (CorEx)

In [44]:
from corextopic import corextopic as ct
from sklearn.feature_extraction.text import CountVectorizer

In [45]:
def create_corex_model(train_corpus, num_topics=1):
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform([' '.join(text) for text in train_corpus])
    
    corex_model = ct.Corex(n_hidden=num_topics, words=vectorizer.get_feature_names_out(), seed=42)
    corex_model.fit(X, words=vectorizer.get_feature_names_out())
    
    return corex_model, vectorizer

In [46]:
corex_model, vectorizer = create_corex_model(train_corpus, num_topics=num_topics)

In [47]:
def get_corex_topic_distribution(article_sentences, corex_model, vectorizer):
    X = vectorizer.transform([' '.join(text) for text in article_sentences])
    topic_distribution = corex_model.transform(X)
    return topic_distribution

In [48]:
corex_topic_distribution = get_corex_topic_distribution(preprocess_data(test_article), corex_model, vectorizer)
important_sentences = get_top_n_sentences(test_article, corex_topic_distribution, n=10)
corex_summary = " ".join(important_sentences)
print("\nGenerated CorEx Abstract:")
print(corex_summary.replace('\n', ' '))


Generated CorEx Abstract:
, chicago , il , usa ) . a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries . in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively . the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% . anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight . snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united states there are also some reports regarding school feed

## Accuracy Assessment

In [50]:
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu

In [51]:
rouge = Rouge()

def evaluate_summary(generated_summary, actual_summary):
    rouge_scores = rouge.get_scores(generated_summary, actual_summary, avg=True)
    
    bleu_score = sentence_bleu([actual_summary.split()], generated_summary.split())    
    return rouge_scores, bleu_score

In [ ]:
def average_scores(rouge_scores_list, bleu_scores_list):
    rouge_avg = {'rouge-1': {'f': 0, 'p': 0, 'r': 0},
                 'rouge-2': {'f': 0, 'p': 0, 'r': 0},
                 'rouge-l': {'f': 0, 'p': 0, 'r': 0}}
    bleu_avg = 0
    
    n = len(rouge_scores_list)
    
    for rouge_scores, bleu_score in zip(rouge_scores_list, bleu_scores_list):
        for key in rouge_avg.keys():
            rouge_avg[key]['f'] += rouge_scores[key]['f']
            rouge_avg[key]['p'] += rouge_scores[key]['p']
            rouge_avg[key]['r'] += rouge_scores[key]['r']
        bleu_avg += bleu_score
        
    for key in rouge_avg.keys():
        rouge_avg[key]['f'] /= n
        rouge_avg[key]['p'] /= n
        rouge_avg[key]['r'] /= n
    
    bleu_avg /= n
    
    return rouge_avg, bleu_avg

In [53]:
rouge_scores, bleu_score = evaluate_summary(summary, train_dataset[0]['abstract'].replace("\n", " "))
print("ROUGE Scores:", rouge_scores)
print("BLEU Score:", bleu_score)

ROUGE Scores: {'rouge-1': {'r': 0.3026315789473684, 'p': 0.33093525179856115, 'f': 0.3161511977591196}, 'rouge-2': {'r': 0.05752212389380531, 'p': 0.058823529411764705, 'f': 0.05816554309906004}, 'rouge-l': {'r': 0.2894736842105263, 'p': 0.31654676258992803, 'f': 0.3024054932917656}}
BLEU Score: 0.03837598605141113


In [66]:
import numpy as np
from tqdm import tqdm

def compute_lda_scores(generate_summary_func, model, vectorizer=None, dictionary=None, n=10):
    rouge_scores_list = []
    bleu_scores_list = []

    for i in tqdm(range(len(test_dataset))):
        article = test_dataset[i]['article']
        actual_summary = test_dataset[i]['abstract'].replace("\n", " ")
        
        generated_summary = " ".join(generate_summary_func(article, model, dictionary)).replace('\n', ' ')

        rouge_scores, bleu_score = evaluate_summary(generated_summary, actual_summary)

        rouge_scores_list.append(rouge_scores)
        bleu_scores_list.append(bleu_score)

    return rouge_scores_list, bleu_scores_list

In [67]:
def average_scores(rouge_scores_list, bleu_scores_list):
    rouge_avg = {'rouge-1': {'f': 0, 'p': 0, 'r': 0},
                 'rouge-2': {'f': 0, 'p': 0, 'r': 0},
                 'rouge-l': {'f': 0, 'p': 0, 'r': 0}}
    bleu_avg = 0
    
    n = len(rouge_scores_list)
    
    for rouge_scores, bleu_score in zip(rouge_scores_list, bleu_scores_list):
        for key in rouge_avg.keys():
            rouge_avg[key]['f'] += rouge_scores[key]['f']
            rouge_avg[key]['p'] += rouge_scores[key]['p']
            rouge_avg[key]['r'] += rouge_scores[key]['r']
        bleu_avg += bleu_score
        
    for key in rouge_avg.keys():
        rouge_avg[key]['f'] /= n
        rouge_avg[key]['p'] /= n
        rouge_avg[key]['r'] /= n
    
    bleu_avg /= n
    
    return rouge_avg, bleu_avg

In [63]:
lda_rouge_scores, lda_bleu_scores = compute_lda_scores(lda_get_top_n_sentences, lda_model, dictionary=dictionary)

  0%|                                                                                         | 0/6658 [00:00<?, ?it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
  0%|                                                                                 | 6/6658 [00:00<04:30, 24.62it/s]C:\Users\ronna\AppData\Loc

In [70]:
lda_avg_rouge, lda_avg_bleu = average_scores(lda_rouge_scores, lda_bleu_scores)

In [71]:
print("LDA - Average ROUGE and BLEU scores:")
print("ROUGE:", lda_avg_rouge)
print("BLEU:", lda_avg_bleu)

LDA - Average ROUGE and BLEU scores:
ROUGE: {'rouge-1': {'f': 0.3347642736560891, 'p': 0.29099570991104895, 'r': 0.4259447185029159}, 'rouge-2': {'f': 0.13646877750930897, 'p': 0.11753033859174543, 'r': 0.1822507674043246}, 'rouge-l': {'f': 0.30466327865932413, 'p': 0.2650233061628383, 'r': 0.38745501151441414}}
BLEU: 0.09292752245553795


In [110]:
def compute_scores(generate_summary_func, topic_distribution_func, model, vectorizer=None, dictionary=None, n=10):
    rouge_scores_list = []
    bleu_scores_list = []

    for i in tqdm(range(len(test_dataset))):
        article = test_dataset[i]['article']
        actual_summary = test_dataset[i]['abstract'].replace("\n", " ")
        
        if vectorizer:
            topic_distribution = topic_distribution_func(preprocess_data(article), model, vectorizer)
        else:
            topic_distribution = topic_distribution_func(preprocess_data(article), model, dictionary)
        generated_summary = " ".join(generate_summary_func(article, topic_distribution, n=10)).replace('\n', ' ')
        
        rouge_scores, bleu_score = evaluate_summary(generated_summary, actual_summary)

        rouge_scores_list.append(rouge_scores)
        bleu_scores_list.append(bleu_score)

    return rouge_scores_list, bleu_scores_list

In [111]:
nmf_rouge_scores, nmf_bleu_scores = compute_scores(get_top_n_sentences, get_nmf_topic_distribution, nmf_model, vectorizer=vectorizer)
nmf_avg_rouge, nmf_avg_bleu = average_scores(nmf_rouge_scores, nmf_bleu_scores)

  0%|                                                                                 | 8/6658 [00:00<06:48, 16.26it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
  0%|▏                                                                               | 12/6658 [00:00<07:04, 15.65it/s]C:\Users\ronna\AppData\Loc

  1%|█                                                                               | 88/6658 [00:05<07:15, 15.10it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 

  3%|██▍                                                                            | 202/6658 [00:12<06:36, 16.28it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
  3%|██▌                                                                            | 217/6658 [00:13<05:31, 19.45it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
  3%|██▋                  

  5%|███▋                                                                           | 316/6658 [00:19<06:59, 15.12it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
  5%|███▊                                                                           | 319/6658 [00:19<06:12, 17.01it/s]C:\Users\ronna\AppData\Loc

  6%|████▉                                                                          | 413/6658 [00:26<05:29, 18.98it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
  6%|█████                                                                          | 424/6658 [00:26<07:13, 14.40it/s]C:\Users\ronna\AppData\Loc

...

 94%|█████████████████████████████████████████████████████████████████████████▌    | 6274/6658 [07:08<00:24, 15.86it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
 94%|█████████████████████████████████████████████████████████████████████████▌    | 6282/6658 [07:09<00:31, 11.96it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
 94%|█████████████████████

 95%|██████████████████████████████████████████████████████████████████████████▍   | 6357/6658 [07:13<00:17, 17.64it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
 96%|██████████████████████████████████████████████████████████████████████████▌   | 6361/6658 [07:13<00:22, 13.28it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ronna\AppData\Loc

 96%|███████████████████████████████████████████████████████████████████████████▏  | 6415/6658 [07:17<00:15, 15.70it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
 96%|███████████████████████████████████████████████████████████████████████████▏  | 6421/6658 [07:17<00:16, 14.68it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
 97%|█████████████████████

 97%|███████████████████████████████████████████████████████████████████████████▊  | 6474/6658 [07:20<00:10, 17.54it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
 97%|███████████████████████████████████████████████████████████████████████████▉  | 6482/6658 [07:21<00:12, 14.50it/s]C:\Users\ronna\AppData\Loc

 99%|█████████████████████████████████████████████████████████████████████████████▏| 6588/6658 [07:29<00:05, 13.99it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
 99%|█████████████████████████████████████████████████████████████████████████████▏| 6593/6658 [07:29<00:03, 17.71it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\ronna\AppData\Loc

In [112]:
lsa_rouge_scores, lsa_bleu_scores = compute_scores(get_top_n_sentences, get_lsa_topic_distribution, lsa_model, vectorizer=vectorizer)
lsa_avg_rouge, lsa_avg_bleu = average_scores(lsa_rouge_scores, lsa_bleu_scores)

  0%|                                                                                 | 7/6658 [00:00<08:01, 13.82it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
  7%|█████▌                                                                         | 468/6658 [00:31<06:41, 15.41it/s]C:\Users\ronna\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
100%|█████████████████████

In [114]:
hdp_rouge_scores, hdp_bleu_scores = compute_scores(get_top_n_sentences, get_hdp_topic_distribution, hdp_model, dictionary=dictionary)
hdp_avg_rouge, hdp_avg_bleu = average_scores(hdp_rouge_scores, hdp_bleu_scores)

100%|██████████████████████████████████████████████████████████████████████████████| 6658/6658 [14:52<00:00,  7.46it/s]


In [115]:
corex_rouge_scores, corex_bleu_scores = compute_scores(get_top_n_sentences, get_corex_topic_distribution, corex_model, vectorizer=vectorizer)
corex_avg_rouge, corex_avg_bleu = average_scores(corex_rouge_scores, corex_bleu_scores)

100%|██████████████████████████████████████████████████████████████████████████████| 6658/6658 [05:52<00:00, 18.89it/s]


In [116]:
print("\nNMF - Average ROUGE and BLEU scores:")
print("ROUGE:", nmf_avg_rouge)
print("BLEU:", nmf_avg_bleu)

print("\nLSA - Average ROUGE and BLEU scores:")
print("ROUGE:", lsa_avg_rouge)
print("BLEU:", lsa_avg_bleu)

print("\nHDP - Average ROUGE and BLEU scores:")
print("ROUGE:", hdp_avg_rouge)
print("BLEU:", hdp_avg_bleu)

print("\nCorEx - Average ROUGE and BLEU scores:")
print("ROUGE:", corex_avg_rouge)
print("BLEU:", corex_avg_bleu)


NMF - Average ROUGE and BLEU scores:
ROUGE: {'rouge-1': {'f': 0.3358000822584319, 'p': 0.27215786819596033, 'r': 0.4862717623594078}, 'rouge-2': {'f': 0.12659233597316474, 'p': 0.10034890070265272, 'r': 0.19789137100625945}, 'rouge-l': {'f': 0.30131913375767005, 'p': 0.24454542889875555, 'r': 0.4356235162153463}}
BLEU: 0.07042092190991009

LSA - Average ROUGE and BLEU scores:
ROUGE: {'rouge-1': {'f': 0.335802429839351, 'p': 0.2721665288423767, 'r': 0.48625263596556995}, 'rouge-2': {'f': 0.12658906287246605, 'p': 0.1003488655324759, 'r': 0.19787117731584375}, 'rouge-l': {'f': 0.3013271310833526, 'p': 0.24455751127746578, 'r': 0.4356168103141936}}
BLEU: 0.07041875964285935

HDP - Average ROUGE and BLEU scores:
ROUGE: {'rouge-1': {'f': 0.30166541389258666, 'p': 0.28587397256499403, 'r': 0.3586961473301454}, 'rouge-2': {'f': 0.09957905674954536, 'p': 0.09406735167812615, 'r': 0.12334645541664681}, 'rouge-l': {'f': 0.2724064927421949, 'p': 0.2585430043155803, 'r': 0.3233826219187103}}
BLEU

In [117]:
print("LDA - Average ROUGE and BLEU scores:")
print("ROUGE:", lda_avg_rouge)
print("BLEU:", lda_avg_bleu)

print("\nNMF - Average ROUGE and BLEU scores:")
print("ROUGE:", nmf_avg_rouge)
print("BLEU:", nmf_avg_bleu)

print("\nLSA - Average ROUGE and BLEU scores:")
print("ROUGE:", lsa_avg_rouge)
print("BLEU:", lsa_avg_bleu)

print("\nHDP - Average ROUGE and BLEU scores:")
print("ROUGE:", hdp_avg_rouge)
print("BLEU:", hdp_avg_bleu)

print("\nCorEx - Average ROUGE and BLEU scores:")
print("ROUGE:", corex_avg_rouge)
print("BLEU:", corex_avg_bleu)

LDA - Average ROUGE and BLEU scores:
ROUGE: {'rouge-1': {'f': 0.3347642736560891, 'p': 0.29099570991104895, 'r': 0.4259447185029159}, 'rouge-2': {'f': 0.13646877750930897, 'p': 0.11753033859174543, 'r': 0.1822507674043246}, 'rouge-l': {'f': 0.30466327865932413, 'p': 0.2650233061628383, 'r': 0.38745501151441414}}
BLEU: 0.09292752245553795

NMF - Average ROUGE and BLEU scores:
ROUGE: {'rouge-1': {'f': 0.3358000822584319, 'p': 0.27215786819596033, 'r': 0.4862717623594078}, 'rouge-2': {'f': 0.12659233597316474, 'p': 0.10034890070265272, 'r': 0.19789137100625945}, 'rouge-l': {'f': 0.30131913375767005, 'p': 0.24454542889875555, 'r': 0.4356235162153463}}
BLEU: 0.07042092190991009

LSA - Average ROUGE and BLEU scores:
ROUGE: {'rouge-1': {'f': 0.335802429839351, 'p': 0.2721665288423767, 'r': 0.48625263596556995}, 'rouge-2': {'f': 0.12658906287246605, 'p': 0.1003488655324759, 'r': 0.19787117731584375}, 'rouge-l': {'f': 0.3013271310833526, 'p': 0.24455751127746578, 'r': 0.4356168103141936}}
BLEU: